<a href="https://colab.research.google.com/github/Tiabet/DACON-code/blob/master/%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bayesian-optimization

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from bayes_opt import BayesianOptimization

# 데이터 불러오기 및 전처리
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

categorical_features = ['대출기간', '주택소유상태', '대출목적']
numerical_features = [col for col in train_df.columns if col not in categorical_features + ['ID', '대출등급', '근로기간']]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

X = preprocessor.fit_transform(train_df.drop(['ID', '대출등급'], axis=1))
y = train_df['대출등급']

# 베이지안 최적화 함수 정의
def rf_eval(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    model = RandomForestClassifier(
        n_estimators=int(n_estimators),
        max_depth=int(max_depth),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        random_state=42
    )
    return cross_val_score(model, X, y, scoring='accuracy', cv=3).mean()

# 하이퍼파라미터 범위 설정
pbounds = {
    'n_estimators': (100, 500),
    'max_depth': (3, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4)
}

optimizer = BayesianOptimization(f=rf_eval, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=2, n_iter=10)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.5687    | 9.367     | 3.852     | 7.856     | 339.5     |


KeyboardInterrupt: 

In [7]:
submission.to_csv('Subsbaseline.csv', index=False)